In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
plt.style.use('default')
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import NMF, PCA
from sklearn.cluster import KMeans
from sklearn.manifold import Isomap, TSNE
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Dense, TimeDistributed, Embedding, GlobalAveragePooling1D, Flatten, SimpleRNN, GRU, Dropout, LSTM, Bidirectional, Lambda
from tensorflow.keras.backend import sum
from tensorflow.keras.utils import plot_model
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import os
import re

# Exercício 1
*Objetivo: explicar as etapas de transformações que são realizadas sobre os dados num processo de classificação com rede neural*

O código abaixo mostra todas as etapas de um processo de treinamento de uma rede neural para análise de sentimentos.

1. Encontre todas as manipulações e transformações que são feitas com o dataset
1. Justifique a relevância de cada uma dessas manipulações, ou, em outras palavras: o que aconteceria se retirássemos cada uma das manipulações?

In [ ]:
df = pd.read_csv('./datasets/IMDB Dataset.csv')
reviews = list(df['review'])

labels = np.array([list(df['sentiment'])]).T
ohe = OneHotEncoder()
y = ohe.fit_transform(labels).toarray()

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(reviews)
sequences = tokenizer.texts_to_sequences(reviews)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded = pad_sequences(sequences,maxlen=200)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(padded, y, test_size=0.2)

In [ ]:
def rede_neural_com_RNN(input_dims, n_dims_out):
  input_layer = Input(shape=(input_dims,))
  x = input_layer
  x = Embedding(1000, 2, name='projecao')(x)
  x = LSTM(30)(x)
  y = Dense(2, activation='softmax', name='classificador')(x)
  return Model(input_layer, y)

rede_neural = rede_neural_com_RNN(200, 2)
rede_neural.compile(optimizer='adam', loss='mse')
plot_model(rede_neural, show_shapes=True, show_layer_activations=True)

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)
history = rede_neural.fit(X_train, y_train, epochs=50, validation_split=0.2, callbacks=es)

In [ ]:
y_est = rede_neural.predict(X_test)
print(classification_report(ohe.inverse_transform(y_test), ohe.inverse_transform(y_est)))

# Exercício 2
*Objetivo: explicar o conceito de atenção*

Em um processo de classificação de texto, é evidente que nem todas as palavras têm a mesma relevância.

1. Na frase: "este filme sobre o lindo mar é terrível", quais palavras são (a) absolutamente irrelevantes, (b) um pouco relevantes e (c) mais relevantes?
1. Na frase: "este filme sobre o terrível mar é lindo", quais palavras são (a) absolutamente irrelevantes, (b) um pouco relevantes e (c) mais relevantes?
1. Ao decidir sobre a relevância de palavras, a ordem de palavras importa?
1. A relevância de uma palavra depende das palavras que a antecedem?
1. A relevância de uma palavra depende das palavras que a sucedem?

# Exercício 3
*Objetivo: entender a implementação de um processo de atenção*

Uma das maneiras de aplicar atenção a uma sequência é multiplicar cada elemento da sequência por um número real, que é alto se aquele elemento recebe mais atenção, e baixo se o elemento não recebe atenção. Após, o resultado pode ser somado. Em outras palavras, *aplicar atenção* significa fazer uma espécie de *média ponderada* das entradas.

Na função abaixo, como esse conceito é aplicado?

In [ ]:
def aplicar_atencao(par):
  estados, atencao = par[0], par[1]
  atencao_aplicada = estados * atencao
  contexto = sum(atencao_aplicada, axis=1)
  return contexto

# Exercício 4
*Objetivo: explicar como a atenção pode ser calculada em uma rede neural*

Analisando a rede neural abaixo, encontre:
1. Quais blocos estão presentes nela, mas não estavam no classificador que fizemos no começo desta aula?
1. Como o valor da *atenção* é calculado?
1. Como a atenção é aplicada?

In [ ]:
def rede_neural_com_atencao(input_dims, n_dims_out):
  input_layer = Input(shape=(input_dims,))
  x = input_layer
  x = Embedding(1000, 2, name='projecao')(x)
  x = LSTM(30, return_sequences=True)(x)

  atencao = TimeDistributed(Dense(1, activation='sigmoid'))(x)
  contexto = Lambda(aplicar_atencao)( [x, atencao] )

  y = Dense(2, activation='softmax', name='classificador')(contexto)
  return Model(input_layer, y), Model(input_layer, atencao)

rede_neural, atencao = rede_neural_com_atencao(200, 2)
rede_neural.compile(optimizer='adam', loss='mse')
plot_model(rede_neural, show_shapes=True, show_layer_activations=True)

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)
history = rede_neural.fit(X_train, y_train, epochs=50, validation_split=0.2, callbacks=es)

In [ ]:
y_est = rede_neural.predict(X_test)
print(classification_report(ohe.inverse_transform(y_test), ohe.inverse_transform(y_est)))

# Exercício 5
*Objetivo: visualizar as saídas da camada de atenção*

Uma informação relevante de uma rede com atenção é o próprio sinal de atenção, pois ele nos informa quais foram as palavras que mais contribuíram para a classificação final.

1. Onde definimos a variável `atencao`? Quando a rede `atencao` foi treinada?
1. As palavras mais relevantes encontradas pela rede fazem sentido em relação ao problema?
1. As palavras mais relevantes são as mesmas em qualquer amostra de texto? Por que?

In [ ]:
at = atencao.predict(X_test)
#print(X_test[0])
#print(at[0])
words = tokenizer.sequences_to_texts(X_test)

In [ ]:
len(words[10].split())
len(at[idx,:])

In [ ]:
idx=15
plt.figure(figsize=(14,2))
n_words = len(words[idx].split())
at_ = at[idx,-n_words:]
plt.plot(at_)
plt.xticks(range(n_words), words[idx].split(), rotation=90)
plt.xlim([0,90])
plt.show()

# Exercício 6
*Objetivo: usar uma rede de atenção com encoder bi-direcional*

Uma propriedade de textos é que, em algumas situações, o significado de palavras depende de palavras que estão mais adiante. Para modelar isso como uma sequência, uma possível solução é (literalmente) inverter o texto, e trabalhar com ele invertido. Nesse caso, precisamos combinar tanto o modelo que vai "para a frente" quanto o que vai "para trás".

No código abaixo:
1. Encontre a camada que lê o texto em sentido *direto*
1. Encontre a camada que lê o texto em sentido *reverso*
1. Encontre a camada que combina as duas informações

In [ ]:
def aplicar_atencao(par):
  estados, atencao = par[0], par[1]
  atencao_aplicada = estados * atencao
  contexto = sum(atencao_aplicada, axis=1)
  return contexto

def rede_neural_com_atencao_bidirecional(input_dims, n_dims_out):
  input_layer = Input(shape=(input_dims,))
  x = input_layer
  x = Embedding(1000, 2, name='projecao')(x)
  x_forward = LSTM(30, return_sequences=True)
  x_backward = LSTM(30, return_sequences=True, go_backwards=True)
  x = Bidirectional(x_forward, backward_layer=x_backward)(x)

  atencao = TimeDistributed(Dense(1, activation='sigmoid'))(x)
  contexto = Lambda(aplicar_atencao)( [x, atencao] )

  y = Dense(2, activation='softmax', name='classificador')(contexto)
  return Model(input_layer, y), Model(input_layer, atencao)

rede_neural, atencao = rede_neural_com_atencao_bidirecional(200, 2)
rede_neural.compile(optimizer='adam', loss='mse')
plot_model(rede_neural, show_shapes=True, show_layer_activations=True, expand_nested=True)

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)
history = rede_neural.fit(X_train, y_train, epochs=50, validation_split=0.2, callbacks=es)

# Exercício 7
*Objetivo: avaliar a rede neural com encoder bi-direcional*

Usando o código abaixo, avalie o desempenho e o sinal de atenção gerado pela nova rede neural.

Houve diferenças evidentes em relação ao caso anterior?

In [ ]:
y_est = rede_neural.predict(X_test)
print(classification_report(ohe.inverse_transform(y_test), ohe.inverse_transform(y_est)))

In [ ]:
at = atencao.predict(X_test)
#print(X_test[0])
#print(at[0])
words = tokenizer.sequences_to_texts(X_test)

In [ ]:
len(words[10].split())
len(at[idx,:])

In [ ]:
idx=88
plt.figure(figsize=(14,2))
n_words = len(words[idx].split())
at_ = at[idx,-n_words:]
plt.plot(at_)
plt.xticks(range(n_words), words[idx].split(), rotation=90)
plt.xlim([0,90])
plt.ylim([0,1])
plt.show()